# Erkennen eines Golden Retrievers mit PyTorch und AlexNet

Importieren der nötigen Libraries:

In [24]:
import torch
from torchvision import models
from torchvision import transforms

Herunterladen des Models und Setzen der Gewichte:

In [25]:
resnet = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V2)

Erstellen eines Transformers, um Bilder für das neuronale Netzwerk nutzbar zu machen (sprich Umwandeln in ein dreidimensionales Array):

In [26]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)])

Laden und Transformieren des Bildes:

In [27]:
from PIL import Image
img = Image.open("../data/p1ch2/bobby.jpg")

![](../data/p1ch2/bobby.jpg)

In [28]:
img_t = preprocess(img)

In [29]:
batch_t = torch.unsqueeze(img_t, 0)

Setzen des Modells in den *eval*-Mode (keine Ahnung was der tut, braucht man hier aber):

In [30]:
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Modell laufen lassen!

In [31]:
out = resnet(batch_t)
out

tensor([[-1.1233e-01, -5.1926e-02,  4.7648e-02,  1.5143e-02, -1.9250e-02,
         -1.1728e-01, -3.3896e-01, -7.1969e-02, -1.1153e-01,  2.3872e-01,
         -4.1327e-02, -2.6325e-01, -7.3170e-02,  1.6063e-01,  1.2323e-02,
         -5.0542e-02, -9.7163e-02, -4.7461e-02,  2.8756e-02,  1.4415e-01,
          4.5259e-02,  9.5436e-02, -2.0497e-01,  1.0033e-01,  8.7252e-03,
          2.1565e-01, -1.3909e-01,  1.0818e-01, -7.9363e-02, -1.6043e-01,
         -3.1740e-02, -2.1645e-01, -3.9362e-02, -2.5555e-01,  1.2436e-01,
          2.6881e-01, -1.5909e-01, -9.4458e-02,  4.5123e-02, -4.2908e-02,
         -1.7030e-01,  1.4350e-01, -2.4532e-02, -1.5386e-01,  1.2039e-01,
          1.9509e-01, -9.1742e-02,  1.5325e-01,  3.7678e-01,  1.0692e-02,
         -1.2965e-01, -3.1077e-02, -4.6631e-02, -1.9775e-01, -1.8386e-01,
         -4.5000e-02,  2.5424e-01,  7.9787e-02, -2.1713e-01,  8.9276e-02,
         -2.1015e-01,  2.3264e-02,  2.8800e-02, -1.5181e-01, -1.4051e-01,
          4.2469e-02,  6.4939e-03, -1.

Das gibt uns jetzt eine Reihe von Werten zurück. Diese sagen aus, wie wahrscheinlich es ist, dass ein bestimmtes Objekt auf diesem Bild vorhanden ist. Da wir aber nicht auswendig wissen, welcher Wert was genau ist, gleichen wir das mit einer Textdatei (welche die *Labels* in der gleichen Reihenfolge gespeichert) ab:

In [32]:
with open('../data/p1ch2/imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

Der folgende Code gibt uns dann die ersten fünf wahrscheinlichsten Objekte (allen voran das richtige: **Golden Retriever**) aus, welche das Modell von vorher erkannt hat:

In [33]:
_, indices = torch.sort(out, descending=True)
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('golden retriever', 74.87625122070312),
 ('Labrador retriever', 1.4267531633377075),
 ('tennis ball', 1.1334877014160156),
 ('toilet tissue, toilet paper, bathroom tissue', 0.21008385717868805),
 ('Bernese mountain dog', 0.20377548038959503)]